In [1]:
# Import necessary libraries
import os
import cv2
import numpy as np
import pandas as pd
import time
import json
from datetime import datetime
import psutil

# Skimage & Mahotas for feature extraction
from skimage.feature import graycomatrix, graycoprops
from skimage.measure import regionprops
import mahotas as mt

# Scikit-learn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, cross_validate, RandomizedSearchCV
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, confusion_matrix, make_scorer
)
from sklearn.svm import SVC
from sklearn.ensemble import (
    RandomForestClassifier, ExtraTreesClassifier
)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline

# Random distributions for hyperparameter optimization
from scipy.stats import randint, uniform

# Add XGBoost
import xgboost as xgb

import joblib

import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay, RocCurveDisplay

# Global Configuration
ROOT_OUTPUT_DIR = "D:\\iate_project\\data\\results\\ml_output"
FEATURE_EXTRACTION_DIR = os.path.join(ROOT_OUTPUT_DIR, 'ml_features', 'ml_feature_extraction')
RESULTS_DIR = os.path.join(ROOT_OUTPUT_DIR, 'ml_classification_results')
MODELS_DIR = os.path.join(RESULTS_DIR, 'ml_models')
PLOTS_DIR = os.path.join(RESULTS_DIR, 'ml_plots')

# Create necessary directories
for d in [ROOT_OUTPUT_DIR, FEATURE_EXTRACTION_DIR, RESULTS_DIR, MODELS_DIR, PLOTS_DIR]:
    os.makedirs(d, exist_ok=True)

# Hyperparameter tuning settings
N_ITER_SEARCH = 15  # Number of parameter settings sampled in RandomizedSearchCV
CV_FOLDS = 5        # Number of cross-validation folds for hyperparameter tuning

# Define base classifiers for hyperparameter optimization
BASE_CLASSIFIERS = {
    'SVM': {
        'model': SVC(probability=True, random_state=42, cache_size=1000),
        'param_dist': {
            'C': uniform(50, 200),
            'gamma': ['scale', 'auto', 0.001, 0.01, 0.1],
            'kernel': ['rbf', 'poly']
        },
        'expects_proba': True,
    },
    'RandomForest': {
        'model': RandomForestClassifier(n_jobs=-1, random_state=42),
        'param_dist': {
            'n_estimators': randint(100, 500),
            'max_depth': [None, 10, 20, 30, 40, 50],
            'min_samples_split': randint(2, 20),
            'min_samples_leaf': randint(1, 10),
            'max_features': ['sqrt', 'log2', None]
        },
        'expects_proba': True,
    },
    'XGBoost': {
        'model': xgb.XGBClassifier(objective='binary:logistic', random_state=42, n_jobs=-1),
        'param_dist': {
            'n_estimators': randint(100, 500),
            'learning_rate': [0.01, 0.05, 0.1, 0.2, 0.3],
            'max_depth': randint(3, 10),
            'subsample': uniform(0.6, 0.4),
            'colsample_bytree': uniform(0.6, 0.4)
        },
        'expects_proba': True,
    },
    'ExtraTrees': {
        'model': ExtraTreesClassifier(n_jobs=-1, random_state=42),
        'param_dist': {
            'n_estimators': randint(100, 600),
            'max_depth': [None, 10, 20, 30, 40, 50],
            'min_samples_split': randint(2, 20),
            'min_samples_leaf': randint(1, 10),
            'max_features': ['sqrt', 'log2', None]
        },
        'expects_proba': True
    },
    'KNN': {
        'model': KNeighborsClassifier(n_jobs=-1),
        'param_dist': {
            'n_neighbors': randint(3, 15),
            'weights': ['uniform', 'distance'],
            'metric': ['euclidean', 'manhattan', 'minkowski']
        },
        'expects_proba': True,
    },
    'DecisionTree': {
        'model': DecisionTreeClassifier(random_state=42),
        'param_dist': {
            'max_depth': [None, 10, 20, 30, 40, 50],
            'min_samples_split': randint(2, 20),
            'min_samples_leaf': randint(1, 10),
            'criterion': ['gini', 'entropy']
        },
        'expects_proba': True
    }
}

# Define feature categories with more accurate naming
FEATURE_CATEGORIES = {
    'color_channel_texture': ['color_texture_'],  # Haralick texture features computed on RGB channels
    'texture': ['glcm_'],                         # Texture features using GLCM on grayscale image
    'shape': ['area', 'perimeter', 'eccentricity', 'extent', 'solidity']  # Shape features
}

# Record resource usage at the beginning
process = psutil.Process(os.getpid())
cpu_usage_before = psutil.cpu_percent(interval=None)
mem_info_before = process.memory_info()
memory_usage_mb_before = mem_info_before.rss / 1024 / 1024

start_time_script = time.time()

# Feature Extraction Configuration
do_feature_extraction = True

# Define path structure based on dataset description
raw_dataset_path = "D:\\iate_project\\data\\raw"
output_dir = FEATURE_EXTRACTION_DIR
os.makedirs(output_dir, exist_ok=True)

df = None

if do_feature_extraction:
    print("\n" + "-"*70)
    print("     STAGE 1: FEATURE EXTRACTION")
    print("-"*70)

    # Timers for each method
    method_times = {
        'color_channel_texture': 0.0,
        'glcm': 0.0,
        'region_props': 0.0
    }

    # Lists to hold extracted data
    feats_list = []
    labels_ = []
    paths_ = []
    splits_ = []

    t_start_fe = time.time()

    # Create storage for class distribution info
    class_counts = {'train': {'normal': 0, 'defect': 0},
                    'test': {'normal': 0, 'defect': 0}}

    # Process image directories based on the specified structure
    classes = ['normal', 'defect']
    processing_types = ['dry', 'honey', 'wet']
    roast_levels = ['dark', 'light', 'medium']

    # Define train/test split ratio (70% train, 30% test)
    train_ratio = 0.7

    # Count total files for progress tracking
    total_files_count = 0
    for class_name in classes:
        for proc_type in processing_types:
            for roast in roast_levels:
                subfolder_path = os.path.join(raw_dataset_path, class_name, proc_type, roast)
                if os.path.exists(subfolder_path):
                    image_files = [f for f in os.listdir(subfolder_path)
                                  if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
                    total_files_count += len(image_files)

    print(f"Total images to process: {total_files_count}")
    print("Starting feature extraction (this may take some time)...")

    # Track processing times and counts for occasional updates
    last_update_time = time.time()
    processed_since_update = 0
    total_processed = 0

    # Walk through the directory structure and extract features
    for class_name in classes:
        for proc_type in processing_types:
            for roast in roast_levels:
                # Construct the path to the current subfolder
                subfolder_path = os.path.join(raw_dataset_path, class_name, proc_type, roast)

                if not os.path.exists(subfolder_path):
                    continue

                # Get list of image files in the current subfolder
                image_files = [f for f in os.listdir(subfolder_path)
                              if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

                if not image_files:
                    continue

                # Shuffle files for random split
                np.random.seed(42)  # For reproducibility
                np.random.shuffle(image_files)

                # Split into train and test
                split_idx = int(len(image_files) * train_ratio)
                train_files = image_files[:split_idx]
                test_files = image_files[split_idx:]

                # Update class distribution counts
                class_counts['train'][class_name] += len(train_files)
                class_counts['test'][class_name] += len(test_files)

                # Process each split
                for split_name, files in [('train', train_files), ('test', test_files)]:
                    batch_start = time.time()
                    batch_size = len(files)

                    # Show batch progress in console
                    print(f"Processing {batch_size} images from {class_name}/{proc_type}/{roast}/{split_name}")

                    for file_ in files:
                        # Construct full path to the image
                        img_path = os.path.join(subfolder_path, file_)

                        # Read and process the image
                        try:
                            # Read color image
                            color_img_ = cv2.imread(img_path, cv2.IMREAD_COLOR)
                            if color_img_ is None:
                                total_processed += 1
                                processed_since_update += 1
                                continue

                            # Convert BGR to RGB
                            color_img_ = cv2.cvtColor(color_img_, cv2.COLOR_BGR2RGB)

                            # Create grayscale image
                            gray_img_ = cv2.cvtColor(color_img_, cv2.COLOR_RGB2GRAY)

                            # Create threshold image (Otsu's method)
                            _, thresh_img_ = cv2.threshold(gray_img_, 0, 255,
                                                        cv2.THRESH_BINARY + cv2.THRESH_OTSU)

                            # Extract features
                            # 1. Color channel texture features (Haralick on RGB channels)
                            t0_ = time.time()
                            color_texture_features = []
                            for d in [1, 2, 3]:  # Distances
                                for i in range(3):  # RGB channels
                                    channel = color_img_[:, :, i]
                                    haralick = mt.features.haralick(channel, distance=d)
                                    color_texture_features.extend(haralick.mean(axis=0).tolist())
                            cost_c = time.time() - t0_
                            method_times['color_channel_texture'] += cost_c

                            # 2. GLCM features for grayscale texture analysis
                            t1_ = time.time()
                            glcm_ = graycomatrix(
                                gray_img_,
                                distances=[1, 2, 3],
                                angles=[0, np.pi/4, np.pi/2, 3*np.pi/4],
                                symmetric=True,
                                normed=True
                            )
                            glcm_features = []
                            props = ['contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation']
                            for prop in props:
                                val_ = graycoprops(glcm_, prop)
                                glcm_features.extend(val_.flatten().tolist())
                            cost_g = time.time() - t1_
                            method_times['glcm'] += cost_g

                            # 3. Region properties - shape features
                            t2_ = time.time()
                            regions = regionprops(thresh_img_.astype(np.uint8))
                            if not regions:
                                region_features = [0] * 5
                            else:
                                props_ = regions[0]
                                region_features = [
                                    props_.area,
                                    props_.perimeter,
                                    props_.eccentricity,
                                    props_.extent,
                                    props_.solidity
                                ]
                            cost_r = time.time() - t2_
                            method_times['region_props'] += cost_r

                            # Combine all features
                            flatten_vec = color_texture_features + glcm_features + region_features
                            feats_list.append(flatten_vec)
                            labels_.append(class_name)
                            paths_.append(os.path.join(class_name, proc_type, roast, file_))
                            splits_.append(split_name)

                        except Exception as e:
                            print(f"Error processing {img_path}: {str(e)}")

                        # Update progress tracking
                        total_processed += 1
                        processed_since_update += 1

                        # Provide occasional progress updates
                        current_time = time.time()
                        if current_time - last_update_time > 60:  # Update every minute
                            percent_done = (total_processed / total_files_count) * 100
                            elapsed = current_time - t_start_fe
                            rate = processed_since_update / (current_time - last_update_time)
                            remaining = (total_files_count - total_processed) / rate if rate > 0 else 0

                            print(f"Progress: {total_processed}/{total_files_count} images ({percent_done:.1f}%), "
                                  f"Rate: {rate:.1f} img/sec, Est. remaining: {remaining/60:.1f} min")

                            last_update_time = current_time
                            processed_since_update = 0

                    # Batch completion summary
                    batch_time = time.time() - batch_start
                    print(f"Completed batch in {batch_time:.1f} seconds ({batch_size/batch_time:.1f} img/sec)")

    print("Feature extraction completed")

    # Check if we got any data
    if not feats_list:
        print("ERROR: No features were extracted.")
        import sys
        sys.exit(-1)

    # Generate feature column names with updated naming for color features
    feat_names = (
        [f'color_texture_{i}' for i in range(13*3*3)] +  # 117 color channel texture features
        [f'glcm_{j}' for j in range(5*3*4)] +           # 60 GLCM texture features
        ['area', 'perimeter', 'eccentricity', 'extent', 'solidity']  # 5 shape features
    )

    # Create DataFrame
    df = pd.DataFrame(feats_list, columns=feat_names)
    df['label'] = labels_
    df['image_path'] = paths_
    df['split'] = splits_

    # Split and scale
    print("Scaling features...")
    train_df = df[df['split'] == 'train']
    test_df = df[df['split'] == 'test']
    scaler = StandardScaler()
    train_mat = scaler.fit_transform(train_df[feat_names])
    test_mat = scaler.transform(test_df[feat_names])
    df.loc[df['split'] == 'train', feat_names] = train_mat
    df.loc[df['split'] == 'test', feat_names] = test_mat

    # Save scaler info
    sc_info = {
        'mean': scaler.mean_.tolist(),
        'scale': scaler.scale_.tolist(),
        'feature_names': feat_names
    }
    with open(os.path.join(output_dir, 'scaler_params.json'), 'w') as f_:
        json.dump(sc_info, f_, indent=4)

    # Summaries
    n_images_ = len(df)
    total_spent_ = time.time() - t_start_fe
    avg_times_ = {k: (method_times[k]/n_images_) for k in method_times}
    method_times['total_time'] = total_spent_
    method_times['average_times'] = avg_times_

    # Save CSV of extracted features
    output_csv_path = os.path.join(output_dir, 'coffee_bean_features.csv')
    df.to_csv(output_csv_path, index=False)

    # Build metadata
    class_dist = df.groupby(['split', 'label']).size()
    class_dist_dict = {
        f"{sp}_{lb}": cnt
        for (sp, lb), cnt in class_dist.items()
    }
    feature_dims = {
        'color_channel_texture': 13*3*3,  # 117 Haralick features on RGB channels
        'glcm': 5*3*4,                    # 60 GLCM features
        'region_props': 5                 # 5 shape features
    }
    meta_ = {
        'extraction_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'processing_time': {
            'total_time': f"{method_times['total_time']:.2f} seconds",
            'method_breakdown': {
                k: f"{method_times[k]:.2f} seconds"
                for k in method_times
                if k not in ['total_time', 'average_times']
            },
            'average_times': {
                k: f"{avg_times_[k]:.4f} seconds/image"
                for k in avg_times_
                if k not in ['total_time', 'average_times']
            }
        },
        'dataset_statistics': {
            'total_samples': len(df),
            'class_distribution': class_dist_dict,
            'feature_dimensions': feature_dims,
            'total_dimensions': sum(feature_dims.values())
        },
        'parameters': {
            'color_channel_texture_distances': [1, 2, 3],
            'glcm_distances': [1, 2, 3],
            'glcm_angles': [0, 45, 90, 135]
        }
    }

    meta_path_ = os.path.join(output_dir, 'extraction_metadata.json')
    with open(meta_path_, 'w') as f_:
        json.dump(meta_, f_, indent=4)

    # Print extraction summary
    print("\n" + "-"*70)
    print("                  FEATURE EXTRACTION SUMMARY")
    print("-"*70)
    print(f"Total samples: {len(df)} images")
    print(f"Feature dimensionality: {meta_['dataset_statistics']['total_dimensions']} features")
    print(f"  - Color channel texture features: {feature_dims['color_channel_texture']} dimensions")
    print(f"  - Texture features (GLCM): {feature_dims['glcm']} dimensions")
    print(f"  - Shape features: {feature_dims['region_props']} dimensions")
    print("\nClass distribution:")

    for s_ in ['train', 'test']:
        dist_s_ = df[df['split'] == s_]['label'].value_counts()
        print(f"  {s_:<5} set => normal: {dist_s_.get('normal', 0):<5}, defect: {dist_s_.get('defect', 0):<5}")

    print(f"\nTotal processing time: {method_times['total_time']:.1f} seconds ({n_images_/method_times['total_time']:.1f} images/sec)")

else:
    print("Skipping feature extraction. Loading features from CSV...")
    csv_path = os.path.join(FEATURE_EXTRACTION_DIR, "coffee_bean_features.csv")
    if os.path.exists(csv_path):
        df = pd.read_csv(csv_path)
        print(f"Loaded {len(df)} samples from {csv_path}")
    else:
        print(f"ERROR: No feature CSV found at {csv_path}. Cannot proceed.")
        import sys
        sys.exit(-1)

if df is None:
    print("ERROR: No data available for classification. Aborting.")
    import sys
    sys.exit(-1)

feat_cols = [c for c in df.columns if c not in ['label', 'image_path', 'split']]
train_df_ = df[df['split'] == 'train']
test_df_ = df[df['split'] == 'test']

X_train = train_df_[feat_cols].values
X_test = test_df_[feat_cols].values
label_map = {'normal': 0, 'defect': 1}
y_train = np.array([label_map[l] for l in train_df_['label']])
y_test = np.array([label_map[l] for l in test_df_['label']])

print(f"Dataset prepared for classification:")
print(f"  - Train set: {len(X_train)} samples")
print(f"  - Test set: {len(X_test)} samples")
print(f"  - Features: {len(feat_cols)} dimensions")

# Define feature combinations to evaluate (updated with new feature category name)
feature_combos = [
    ['color_channel_texture'],
    ['texture'],
    ['shape'],
    ['color_channel_texture', 'texture'],
    ['color_channel_texture', 'shape'],
    ['texture', 'shape'],
    ['color_channel_texture', 'texture', 'shape']
]

# Setup for results collection
all_results = {}
best_model_script = {
    'score': -1,
    'combo': None,
    'clf_name': None,
    'model': None,
    'results': None,
    'features': None,
    'best_params': None
}

print("\nEvaluating feature combinations with hyperparameter tuning...")
print(f"This will test {len(feature_combos)} feature combinations × {len(BASE_CLASSIFIERS)} classifiers")
print(f"Each classifier will be optimized using RandomizedSearchCV with {N_ITER_SEARCH} iterations")

# Set up evaluation progress tracking
total_evals = len(feature_combos) * len(BASE_CLASSIFIERS)
start_eval_time = time.time()
completed_evals = 0

# Loop through each feature combination
for combo_ in feature_combos:
    # Select features for this combination
    idx_sel_ = []
    dims_ = {}

    # Select feature indices for current combination
    for cat_ in combo_:
        cat_cols_ = []
        for prefix_ in FEATURE_CATEGORIES[cat_]:
            if prefix_.endswith('_'):
                found_ = [i for i, c in enumerate(feat_cols) if c.startswith(prefix_)]
            else:
                found_ = [i for i, c in enumerate(feat_cols) if c == prefix_]
            cat_cols_.extend(found_)
        dims_[cat_] = len(cat_cols_)
        idx_sel_.extend(cat_cols_)

    # Extract selected features
    X_train_sel_ = X_train[:, idx_sel_]
    X_test_sel_ = X_test[:, idx_sel_]
    sel_names_ = [feat_cols[i] for i in idx_sel_]

    combo_name_ = '+'.join(combo_)
    all_results[combo_name_] = {}

    # Compact feature info log
    feature_info = ", ".join([f"{c}: {dims_[c]}" for c in combo_])
    print(f"\nEvaluating feature combination: '{combo_name_}' ({feature_info})")

    # No feature selection - using all features
    print("Using all features (no feature selection)")

    # All features are kept - create a mask of all ones
    selected_mask = np.ones(len(sel_names_), dtype=bool)

    # Evaluate each classifier with this feature combination
    for clf_name_, clf_info_ in BASE_CLASSIFIERS.items():
        eval_start = time.time()

        print(f"Optimizing {clf_name_} with hyperparameter tuning...")

        # Create a pipeline with just the classifier
        pipeline_ = Pipeline([
            ('classifier', clf_info_['model'])
        ])

        # Setup parameter distributions for the pipeline
        param_dist = {}
        for param, value in clf_info_['param_dist'].items():
            param_dist[f'classifier__{param}'] = value

        # Create RandomizedSearchCV for hyperparameter tuning
        random_search = RandomizedSearchCV(
            estimator=pipeline_,
            param_distributions=param_dist,
            n_iter=N_ITER_SEARCH,
            cv=CV_FOLDS,
            scoring='f1',
            n_jobs=-1,
            random_state=42,
            verbose=0
        )

        # Fit RandomizedSearchCV to find best parameters
        random_search.fit(X_train_sel_, y_train)

        # Get best model and parameters
        best_model = random_search.best_estimator_
        best_params = random_search.best_params_

        # Print best parameters in a readable format
        print(f"Best parameters for {clf_name_}:")
        for param, value in best_params.items():
            print(f"  {param.replace('classifier__', '')}: {value}")

        # Cross-validation with best model
        cv_metrics_ = {}
        splitter_ = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
        scoring_ = {
            'accuracy': 'accuracy',
            'precision': make_scorer(precision_score, zero_division=0),
            'recall': 'recall',
            'f1': 'f1',
            'roc_auc': 'roc_auc'
        }
        cv_results_ = cross_validate(
            best_model, X_train_sel_, y_train,
            scoring=scoring_, cv=splitter_, n_jobs=-1, return_train_score=True
        )

        # Process CV results
        for metric, scores in [(k.replace('test_', ''), cv_results_[k])
                               for k in cv_results_ if k.startswith('test_')]:
            cv_metrics_[metric] = {
                'mean': float(scores.mean()),
                'std': float(scores.std()),
                'values': scores.tolist()
            }

        # Predict test set
        y_pred_ = best_model.predict(X_test_sel_)
        y_proba_ = None
        if clf_info_['expects_proba']:
            y_proba_ = best_model.predict_proba(X_test_sel_)[:, 1]

        # Compute test metrics
        metrics_ = {
            'accuracy': accuracy_score(y_test, y_pred_),
            'precision': precision_score(y_test, y_pred_),
            'recall': recall_score(y_test, y_pred_),
            'f1': f1_score(y_test, y_pred_),
            'roc_auc': roc_auc_score(y_test, y_proba_) if y_proba_ is not None else None
        }

        # Get feature importances if available
        final_clf_ = best_model.named_steps['classifier']
        feat_imp_ = None
        if hasattr(final_clf_, 'feature_importances_'):
            feat_imp_ = final_clf_.feature_importances_
        elif hasattr(final_clf_, 'coef_'):
            feat_imp_ = np.abs(final_clf_.coef_[0])

        # Record total time
        total_t_ = time.time() - eval_start

        # Store results
        result_dict_ = {
            'model_name': clf_name_,
            'cross_validation': cv_metrics_,
            'test_performance': metrics_,
            'timing': {
                'total_time': total_t_
            },
            'feature_importance': feat_imp_.tolist() if feat_imp_ is not None else None,
            'best_params': best_params,
            'selected_features': selected_mask.tolist() if hasattr(selected_mask, 'tolist') else None
        }
        all_results[combo_name_][clf_name_] = result_dict_

        # Track the best model by F1 score
        cur_f1_ = metrics_['f1']
        if cur_f1_ > best_model_script['score']:
            best_model_script['score'] = cur_f1_
            best_model_script['combo'] = combo_
            best_model_script['clf_name'] = clf_name_
            best_model_script['model'] = best_model
            best_model_script['results'] = result_dict_
            best_model_script['features'] = sel_names_
            best_model_script['best_params'] = best_params

        # Update progress
        completed_evals += 1
        elapsed = time.time() - start_eval_time
        rate = completed_evals / elapsed if elapsed > 0 else 0
        remaining = (total_evals - completed_evals) / rate if rate > 0 else 0

        # Print result with clear formatting, highlight best so far
        is_best = (best_model_script['clf_name'] == clf_name_ and
                   best_model_script['combo'] == combo_)

        best_marker = "★ " if is_best else "  "

        print(f"{best_marker}{clf_name_:15} | "
              f"F1: {metrics_['f1']:.4f} | "
              f"Acc: {metrics_['accuracy']:.4f} | "
              f"Prec: {metrics_['precision']:.4f} | "
              f"Rec: {metrics_['recall']:.4f} | "
              f"Time: {total_t_:.1f}s")

print("All models evaluated successfully.")

# Create a summary table
summary_table = []
for combo_key, model_dict in all_results.items():
    for model_k, model_res in model_dict.items():
        summary_table.append({
            'Feature Combo': combo_key,
            'Model': model_k,
            'Accuracy': f"{model_res['test_performance']['accuracy']:.4f}",
            'Precision': f"{model_res['test_performance']['precision']:.4f}",
            'Recall': f"{model_res['test_performance']['recall']:.4f}",
            'F1 Score': f"{model_res['test_performance']['f1']:.4f}",
            'ROC AUC': (
                f"{model_res['test_performance']['roc_auc']:.4f}"
                if model_res['test_performance']['roc_auc'] is not None
                else 'N/A'
            )
        })

summary_df = pd.DataFrame(summary_table)

# Print highly visible results banner
print(f"\nBEST MODEL: {best_model_script['clf_name']} with {'+'.join(best_model_script['combo'])}")
print(f"F1 SCORE: {best_model_script['score']:.4f}")
print(f"ACCURACY: {best_model_script['results']['test_performance']['accuracy']:.4f}")
print(f"PRECISION: {best_model_script['results']['test_performance']['precision']:.4f}")
print(f"RECALL: {best_model_script['results']['test_performance']['recall']:.4f}")

# Show best parameters
print("\nBEST HYPERPARAMETERS:")
for param, value in best_model_script['best_params'].items():
    print(f"  {param.replace('classifier__', '')}: {value}")

# Show top 3 models
print("\nTOP 3 MODELS BY F1 SCORE:")
top_models = sorted(
    [(combo, model, results['test_performance']['f1'])
     for combo, models in all_results.items()
     for model, results in models.items()],
    key=lambda x: x[2], reverse=True
)[:3]

for i, (combo, model, f1) in enumerate(top_models, 1):
    result = all_results[combo][model]['test_performance']
    print(f"{i}. {model} with {combo}")
    print(f"   F1: {f1:.4f} | Acc: {result['accuracy']:.4f} | "
          f"Prec: {result['precision']:.4f} | Rec: {result['recall']:.4f}")

# Save results as JSON
out_json_ = os.path.join(RESULTS_DIR, 'classification_results.json')
with open(out_json_, 'w') as f_:
    json.dump(all_results, f_, indent=4)

# Save best model
model_path_ = os.path.join(MODELS_DIR, 'best_model.joblib')
joblib.dump(best_model_script['model'], model_path_)

# Save feature info
feature_info_path = os.path.join(MODELS_DIR, 'feature_info.json')
feature_info = {
    'feature_combination': best_model_script['combo'],
    'features': best_model_script['features']
}
with open(feature_info_path, 'w') as f_:
    json.dump(feature_info, f_, indent=4)

# Save a detailed report
best_report_ = {
    'model_summary': {
        'name': best_model_script['clf_name'],
        'feature_combination': '+'.join(best_model_script['combo']),
        'total_features': len(best_model_script['features']),
        'best_hyperparameters': best_model_script['best_params']
    },
    'performance_metrics': best_model_script['results']['test_performance'],
    'timing_information': best_model_script['results']['timing'],
    'feature_importance': (
        dict(zip(
            best_model_script['features'],
            best_model_script['results']['feature_importance']
        )) if best_model_script['results']['feature_importance'] is not None else None
    )
}
report_path_ = os.path.join(RESULTS_DIR, 'detailed_report.json')
with open(report_path_, 'w') as f_:
    json.dump(best_report_, f_, indent=4)

print("\nClassification results and best model saved to disk.")

# Generate best model plots
# Select features for best model
idx_sel_ = []
for cat_ in best_model_script['combo']:
    for prefix_ in FEATURE_CATEGORIES[cat_]:
        if prefix_.endswith('_'):
            found_ = [i for i, c in enumerate(feat_cols) if c.startswith(prefix_)]
        else:
            found_ = [i for i, c in enumerate(feat_cols) if c == prefix_]
        idx_sel_.extend(found_)
X_test_best_ = X_test[:, idx_sel_]

# Get predictions
y_pred_best_ = best_model_script['model'].predict(X_test_best_)
y_proba_best_ = None
if hasattr(best_model_script['model'].named_steps['classifier'], 'predict_proba'):
    y_proba_best_ = best_model_script['model'].predict_proba(X_test_best_)[:,1]

# Plot confusion matrix
print("Generating confusion matrix...")
cm_path_ = os.path.join(PLOTS_DIR, 'best_model_confusion_matrix.png')
cm_ = confusion_matrix(y_test, y_pred_best_)
classes_ = ['normal', 'defect']
disp_ = ConfusionMatrixDisplay(cm_, display_labels=classes_)
fig_, ax_ = plt.subplots(figsize=(8, 7))
disp_.plot(ax=ax_, cmap=plt.cm.Blues, colorbar=False)
plt.title(f"Confusion Matrix - {best_model_script['clf_name']} with {'+'.join(best_model_script['combo'])}")
plt.tight_layout()
plt.savefig(cm_path_)
plt.close()

# Print confusion matrix values
tn, fp, fn, tp = cm_.ravel()
print("\nConfusion Matrix:")
print(f"  True Negatives (normal correctly predicted): {tn}")
print(f"  False Positives (normal incorrectly as defect): {fp}")
print(f"  False Negatives (defect incorrectly as normal): {fn}")
print(f"  True Positives (defect correctly predicted): {tp}")

# Plot ROC curve if probability estimates available
if y_proba_best_ is not None:
    print("Generating ROC curve...")
    roc_path_ = os.path.join(PLOTS_DIR, 'best_model_roc_curve.png')
    disp_ = RocCurveDisplay.from_predictions(y_test, y_proba_best_)
    fig_ = disp_.figure_
    plt.title(f"ROC Curve - {best_model_script['clf_name']} with {'+'.join(best_model_script['combo'])}")
    plt.tight_layout()
    plt.savefig(roc_path_)
    plt.close()

# Plot feature importances if available
if best_model_script['results']['feature_importance'] is not None:
    print("Generating feature importance visualization...")
    fi_path_ = os.path.join(PLOTS_DIR, 'best_model_feature_importances.png')
    fi_abs_ = np.abs(np.array(best_model_script['results']['feature_importance']))
    idx_sorted_ = np.argsort(fi_abs_)[::-1]
    top_n = min(15, len(fi_abs_))
    fi_sorted_ = fi_abs_[idx_sorted_][:top_n]
    names_sorted_ = [best_model_script['features'][i] for i in idx_sorted_[:top_n]]

    fig_, ax_ = plt.subplots(figsize=(10, 8))
    y_pos_ = np.arange(len(fi_sorted_))
    ax_.barh(y_pos_, fi_sorted_[::-1], align='center', color='skyblue')
    ax_.set_yticks(y_pos_)
    ax_.set_yticklabels(names_sorted_[::-1])
    ax_.invert_yaxis()
    ax_.set_xlabel('Importance')
    ax_.set_title(f'Feature Importances - Top {top_n}')
    plt.tight_layout()
    plt.savefig(fi_path_)
    plt.close()

    # Print top features
    print("\nTop 5 Most Important Features:")
    for i in range(min(5, top_n)):
        print(f"  {i+1}. {names_sorted_[i]}: {fi_sorted_[i]:.4f}")

# Calculate final resource usage
cpu_usage_after = psutil.cpu_percent(interval=None)
mem_info_after = process.memory_info()
memory_usage_mb_after = mem_info_after.rss / 1024 / 1024

total_time_script = time.time() - start_time_script

print(f"Total execution time: {total_time_script:.1f} seconds ({total_time_script/60:.1f} minutes)")
print(f"Memory usage: {memory_usage_mb_before:.1f}MB → {memory_usage_mb_after:.1f}MB")
print(f"All results saved to: {RESULTS_DIR}")


----------------------------------------------------------------------
     STAGE 1: FEATURE EXTRACTION
----------------------------------------------------------------------
Total images to process: 5400
Starting feature extraction (this may take some time)...
Processing 280 images from normal/dry/dark/train
Completed batch in 48.2 seconds (5.8 img/sec)
Processing 120 images from normal/dry/dark/test
Progress: 347/5400 images (6.4%), Rate: 5.8 img/sec, Est. remaining: 14.6 min
Completed batch in 21.9 seconds (5.5 img/sec)
Processing 280 images from normal/dry/light/train
Progress: 648/5400 images (12.0%), Rate: 5.0 img/sec, Est. remaining: 15.8 min
Completed batch in 57.0 seconds (4.9 img/sec)
Processing 120 images from normal/dry/light/test
Completed batch in 24.7 seconds (4.9 img/sec)
Processing 280 images from normal/dry/medium/train
Progress: 953/5400 images (17.6%), Rate: 5.1 img/sec, Est. remaining: 14.6 min
Completed batch in 52.8 seconds (5.3 img/sec)
Processing 120 images fr